In [26]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from datetime import datetime

In [27]:
url = 'https://www.mercadolivre.com.br/ofertas?container_id=MLBB779362-1&page='

data = requests.get(url).text

In [28]:
type(data)

str

In [29]:
with open('mercadolivreoferdia.html', 'w', encoding='utf-8') as f:
    f.write(data)

In [30]:
# retorna títulos dos produtos nas tags
def get_item_title(doc):
    title_tags = doc.find_all('p', class_='promotion-item_title')
    titles = []
    for tags in title_tags:
        titles.append(tags.text)
    
    return titles

In [48]:
import re

def get_item_price(doc):
    price_tags = doc.find_all('p', class_='promotion-item_price')
    prices = []
    for tag in price_tags:
        price_text = tag.get_text(strip=True)
        price_match = re.search(r'(\d+[\.,]?\d*)', price_text)
        if price_match:
            price_value = price_match.group(1)
            prices.append(price_value)
        else:
            prices.append(None)
    
    return prices


In [32]:
def get_doc(url):
    response = requests.get(url)
    if response.status_code != 200:
        raise Exception("Deu ruim. Resposta esquisiiiita...")
    doc = BeautifulSoup(response.text, 'html.parser')
    
    return doc

In [42]:
def scrape_multiple_pages(n):
    titles, prices = [], []

    for page in range(1, n+1):
        doc = get_doc(url + str(page))
        titles.extend(get_item_title(doc))
        prices.extend(get_item_price(doc))

        # Imprimir alguns exemplos dos preços
        print("Exemplos dos preços:", prices[:5])
    
    itens = {
        'TITLE' : titles,
        'PRICE' : prices
    }

    return pd.DataFrame(itens)


In [49]:
df_mercado_livre_ofer_dia = scrape_multiple_pages(10)
df_mercado_livre_ofer_dia['current_timestamp'] = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
df_mercado_livre_ofer_dia['PRICE'] = df_mercado_livre_ofer_dia['PRICE'].str.extract(r'(\d+[\.,]?\d*)')
df_mercado_livre_ofer_dia.head()

Exemplos dos preços: []
Exemplos dos preços: []
Exemplos dos preços: []
Exemplos dos preços: []
Exemplos dos preços: []
Exemplos dos preços: []
Exemplos dos preços: []
Exemplos dos preços: []
Exemplos dos preços: []
Exemplos dos preços: []


AttributeError: Can only use .str accessor with string values!